In [4]:
#importing the essentials
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import scipy as sp
import warnings
import datetime
warnings.filterwarnings("ignore")
%matplotlib inline

In [27]:
#trying out Folium
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [6]:
#I had to install the seaborn package and folium package
#! conda install seaborn -y
#! conda install folium -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.14.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/studio-lab-user/.conda/envs/default

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.5.0               |     pyhd8ed1ab_0          26 KB  conda-forge
    brotlipy-0.7.0             |py39hb9d737c_1004         342 KB  conda-forge
    charset-normalizer-2.1.1   |     pyhd8ed1ab_0          36 KB  conda-forge
    cryptography-37.0.4        |   py39hd97740a_0         1.5 MB  conda-forge
    folium-0.12.1.post1        |     pyhd8ed1ab_1          64 KB  conda-forge
    idna-3.3                   |     pyhd8ed1ab_0          55 KB  conda-forge
    pyopenssl-22.0.0           |     pyhd8ed1ab_0          49 K

In [14]:
#importing boston crime data.
#source : https://www.kaggle.com/code/alexisbcook/interactive-maps/data
crime_df = pd.read_csv('crime.csv', encoding='latin-1')

In [15]:
crime_df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [17]:
# 319073 rows and 17 columns
crime_df.shape

(319073, 17)

In [20]:
crime_df['YEAR'].value_counts()

2017    100886
2016     99114
2018     65685
2015     53388
Name: YEAR, dtype: int64

In [22]:
#For the purpose of this submission, let's focus on the crimes that happened in 2015
crime_df = crime_df[crime_df['YEAR']==2015]

In [43]:
# the columns Lat, Long and DISTRICT has missing value. Final map view would throw an error if there are NaN values
crime_df.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

In [44]:
# Not essential but focus on Larceny, auto theft, Robbery, Criminal Harassment and Manslaughter for this weeks general assembly submission 
crime_df = crime_df[crime_df.OFFENSE_CODE_GROUP.isin([
    'Larceny', 'Auto Theft', 'Robbery', 'Criminal Harassment', 
    'Manslaughter'])]

In [45]:
crime_df.tail(2)

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
319044,I110551302-00,623,Larceny,LARCENY SHOPLIFTING $50 TO $199,D4,171,NaN,2015-07-22 22:00:00,2015,7,Wednesday,22,Part One,HARRISON AVE,42.335560,-71.074364,"(42.33555954, -71.07436364)"
319055,I100340225-00,339,Robbery,ROBBERY - UNARMED - STREET,A1,77,NaN,2015-07-27 10:47:00,2015,7,Monday,10,Part One,BOWDOIN SQ,42.361645,-71.062299,"(42.36164502, -71.06229949)"


In [46]:
# Create a map
mp = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
for idx, row in crime_df.iterrows():
    Marker([row['Lat'], row['Long']]).add_to(mp)

# Display the map
mp

In [47]:
#The above seems like a lot of pointers on the map. Let's limit it

In [49]:
crime_df2 = crime_df[((crime_df.OFFENSE_CODE_GROUP == 'Robbery') & \
                            (crime_df.HOUR.isin(range(9,18))))]

In [51]:
# Create a map
map2 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
for idx, row in crime_df2.iterrows():
    Marker([row['Lat'], row['Long']]).add_to(map2)

# Display the map
map2